In [1]:
import sys
import pandas as pd
sys.path.append("..")

# news scraping
from src.zinfo.news_scraper import NewsScraper

# clustering
from src.zinfo.article_clustering import get_vectorized_titles
from src.zinfo.article_clustering import cluster_articles

# article selecting
from src.zinfo.article_selector import get_best_article_all_clusters

In [2]:
# scraper = NewsScraper('eeeaefaae3c14737bc08e252a6e1991b')

In [3]:
# trending_news = scraper.get_trending_articles_today(5)

In [8]:
trending_news = pd.read_csv("test.csv")
trending_news = trending_news.drop_duplicates(subset=['title'])

In [9]:
clusters = cluster_articles(trending_news)

clustering articles
eps_val: 0.28
min_samples: 2


In [10]:
summarized_news = get_best_article_all_clusters(clusters, trending_news)

selecting best articles out of each cluster


In [ ]:
summarized_news